In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("events_log.csv")
data

,uuid,timestamp,session_id,group,action,checkin,page_id,n_results,result_position
0,00000736167c507e8ec225bd9e71f9e5,2.016030e+13,78245c2c3fba013a,b,searchResultPage,NaN,cbeb66d1bc1f1bc2,5.0,NaN
1,00000c69fe345268935463abbfa5d5b3,2.016031e+13,c559c3be98dca8a4,a,searchResultPage,NaN,eb658e8722aad674,10.0,NaN
2,00003bfdab715ee59077a3670331b787,2.016030e+13,760bf89817ce4b08,a,checkin,30.0,f99a9fc1f7fdd21e,NaN,NaN
3,0000465cd7c35ad2bdeafec953e08c1a,2.016030e+13,fb905603d31b2071,a,checkin,60.0,e5626962a6939a75,NaN,10.0
4,000050cbb4ef5b42b16c4d2cf69e6358,2.016030e+13,c2bf5e5172a892dc,a,checkin,30.0,787dd6a4c371cbf9,NaN,NaN
...,...,...,...,...,...,...,...,...,...
400160,ffffa98532e154efa821aec083f92674,2.016030e+13,50d4c8dce4b959c3,b,checkin,120.0,be6ad202740eef90,NaN,2.0
400161,ffffbe4347aa5c4facc64fa4a26e73e4,2.016031e+13,056fc33556fcd8c0,b,checkin,10.0,c32a6f39204b8c83,NaN,1.0
400162,ffffc4bd7f8e53a1a86b38874c6728b1,2.016030e+13,f8487a76f255ee62,b,searchResultPage,NaN,b24925ad79226f74,20.0,NaN
400163,ffffe6cffc8d510db61c3f3dce918849,2.016031e+13,daa2d7654ca30c15,b,searchResultPage,NaN,c5d5fe35920a8833,20.0,NaN


## Format data
### Delete not necessary columns

In [4]:
data.drop("uuid",axis=1,inplace=True)

### Format timestamp column to 'YYYY-MM-DD hh:mm:ss'

In [5]:
data.timestamp[0]

20160301103842.0

In [6]:
def format_timestamp(timestamp):
    timestamp = str(timestamp)
    YYYY,MM,DD,hh,mm,ss = timestamp[:4], timestamp[4:6], timestamp[6:8], timestamp[8:10], timestamp[10:12], timestamp[12:]
    return YYYY + '-' + MM + '-' + DD + ' ' + hh + ':' + mm + ':' + ss

In [8]:
data['timestamp'] = data['timestamp'].apply(lambda x : '{:.0f}'.format(float(x)))
data['timestamp'] = data['timestamp'].apply(format_timestamp)
data['timestamp'] = pd.to_datetime(data['timestamp'], format="%Y-%m-%d %H:%M:%S")

TypeError: float() argument must be a string or a number, not 'Timestamp'

### Extracting the date from the timestamp for the task purpose

In [12]:
data["Date"]=data.timestamp.apply(lambda d:d.date())
data.head()

,timestamp,session_id,group,action,checkin,page_id,n_results,result_position,Date
0,2016-03-01 10:38:42,78245c2c3fba013a,b,searchResultPage,NaN,cbeb66d1bc1f1bc2,5.0,NaN,2016-03-01
1,2016-03-07 00:52:26,c559c3be98dca8a4,a,searchResultPage,NaN,eb658e8722aad674,10.0,NaN,2016-03-07
2,2016-03-02 14:53:05,760bf89817ce4b08,a,checkin,30.0,f99a9fc1f7fdd21e,NaN,NaN,2016-03-02
3,2016-03-02 22:27:01,fb905603d31b2071,a,checkin,60.0,e5626962a6939a75,NaN,10.0,2016-03-02
4,2016-03-02 09:50:23,c2bf5e5172a892dc,a,checkin,30.0,787dd6a4c371cbf9,NaN,NaN,2016-03-02


## Task 1: What is the daily overall clickthrough rate? How does it vary between the groups?

### Daily overall clickthrough rate
<ol>
<li>Select data where action = 'checkin' #We only care about sessions where users have clicked on a result</li>
<li>Select distinct session_id by removing duplicates</li>
<li>Group by date and count the number of sessions for each date</li>
<li>Then take the mean which represent the daily overall clickthrough rate </li>
</ol>

In [38]:
temp = data[(data.action == 'checkin')].drop_duplicates(subset='session_id').groupby('Date').size().reset_index(name='counts')
temp

,Date,counts
0,2016-03-01,3234
1,2016-03-02,3290
2,2016-03-03,3301
3,2016-03-04,2805
4,2016-03-05,2201
5,2016-03-06,2542
6,2016-03-07,3219
7,2016-03-08,2703


In [39]:
#daily overall clickthough rate 
temp.mean()

counts    2911.875
dtype: float64

## Task 2: Which results do people tend to try first? How does it change day-to-day? 

### Which results do people tend to try first?
<ol>
<li>Remove data where result_position are none</li>
<li>Group by result_position</li>
<li>Count the number of elements per group and sort in decreasing order</li>
</ol>

In [70]:
temp2=data[data["result_position"].isnull()==False].groupby("result_position").size().reset_index(name="count")
temp2

,result_position,count
0,1.0,149354
1,2.0,31779
2,3.0,16234
3,4.0,8698
4,5.0,5334
...,...,...
188,3895.0,5
189,3955.0,1
190,3972.0,7
191,4087.0,6


In [74]:
temp2.sort_values(by=["count","result_position"],ascending=False)
temp2

,result_position,count
0,1.0,149354
1,2.0,31779
2,3.0,16234
3,4.0,8698
4,5.0,5334
...,...,...
188,3895.0,5
189,3955.0,1
190,3972.0,7
191,4087.0,6


## Task 3: What is the daily overall zero results rate? How does it vary between the groups?

### Daily overall zero results rate
<ol>
<li>Select data where action = 'searchResultPage' and number of results = 0</li>
<li>Group data by date and count</li>
<li>Then take the mean</li>
</ol>

In [80]:
temp3=data[(data["action"]=="searchResultPage") & (data["n_results"]==0)].groupby(["group","Date"]).size().reset_index(name="count")
temp3

,group,Date,count
0,a,2016-03-01,2350
1,a,2016-03-02,2524
2,a,2016-03-03,2371
3,a,2016-03-04,2068
4,a,2016-03-05,1535
5,a,2016-03-06,1599
6,a,2016-03-07,2316
7,a,2016-03-08,2139
8,b,2016-03-01,1097
9,b,2016-03-02,1065


In [87]:
temp3.groupby("group").mean()

,count
group,
a,2112.750
b,1028.125
